In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
# Set up the Selenium driver
driver = webdriver.Chrome()
url = "https://www.gaadibazaar.in/used-second-hand-cars-in-india"  # Replace with the actual URL
driver.get(url)

wait = WebDriverWait(driver, 10)

# Function to click the "Show More Cars" button
def click_show_more_button():
    try:
        show_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='load_more_pagination pointer' and contains(text(), 'Show more cars')]")))
        show_more_button.click()
        return True
    except:
        return False

car_data_list = []
# Function to extract data from a car listing element
def extract_car_data(car_data_element):
    car_name = car_data_element.find_element(By.TAG_NAME, "h2").text
    # car_model = car_data_element.find_element(By.CLASS_NAME, "flex-row").find_element(By.CLASS_NAME, "d-blue").text
    car_model_element = car_data_element.find_element(By.CLASS_NAME, "flex-row")
    car_model = car_model_element.text.strip()
    car_details = car_data_element.find_elements(By.CLASS_NAME, "medium")

    if len(car_details) >= 4:
        car_year = car_details[0].text
        car_mileage = car_details[1].text
        car_fuel_type = car_details[2].text
        car_owner = car_details[3].text
    else:
        car_year = car_mileage = car_fuel_type = car_owner = "N/A"

    car_price = car_data_element.find_element(By.CLASS_NAME, "bold").text
    car_data_list.append([car_name, car_model, car_year, car_mileage, car_fuel_type, car_owner, car_price])

    # print(f"Car Name: {car_name}")
    # print(f"Car Model: {car_model}")
    # print(f"Year: {car_year}")
    # print(f"Mileage: {car_mileage}")
    # print(f"Fuel Type: {car_fuel_type}")
    # print(f"Owner: {car_owner}")
    # print(f"Price: {car_price}")
    # print("\n")

# Initially, extract data from the visible car listings
car_data_elements = driver.find_elements(By.CLASS_NAME, "card-list")
for car_data_element in car_data_elements:
    extract_car_data(car_data_element)

# Click the "Show More Cars" button up to 10 times
click_count = 0
while click_count <= 30:
    print(click_count)
    if not click_show_more_button():
        break  # If the button is not available, exit the loop
    click_count += 1
    # Wait for new data to load
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "card-list")))

    # Extract data from the newly loaded car listings
    car_data_elements = driver.find_elements(By.CLASS_NAME, "card-list")
    for car_data_element in car_data_elements:
        extract_car_data(car_data_element)

# Close the browser
driver.quit()
# Create a DataFrame from the list
df = pd.DataFrame(car_data_list, columns=["Car Name", "Car Model", "Year", "Mileage", "Fuel Type", "Owner", "Price"])

0
1
2
3
4


In [13]:
df.to_csv('gaadi.csv')
df.head()

,Car Name,Car Model,Year,Mileage,Fuel Type,Owner,Price
0,HYUNDAI CRETA,1.6 SX PLUS PETROL,2018,9000,Petrol,1st Owner,₹9L
1,MAHINDRA XUV500,W6,2014,81000,Diesel,1st Owner,₹5.8L
2,MARUTI SUZUKI ALTO,STD BS III,2006,80000,Petrol,1st Owner,₹1.9L
3,TOYOTA Corolla Altis,1.8 VL,2012,28500,Petrol,2nd Owner,₹3.5L
4,MARUTI SUZUKI WAGON R,LXI,2017,25000,Petrol,1st Owner,₹3.5L


In [14]:
df.shape,df.isnull().sum()

((231, 7),
 Car Name     0
 Car Model    0
 Year         0
 Mileage      0
 Fuel Type    0
 Owner        0
 Price        0
 dtype: int64)